### Let's create a table called tracks_by_year as our new table

In [6]:
%%cql create table music.tracks_by_year (dummy text, year int, track_count int, primary key (dummy, track_count) ) with clustering order by (track_count desc)

### Register the table so it uses the correct input format
- This will work out of the box in a future release

In [7]:
sqlContext.sql("""CREATE TEMPORARY TABLE tmp_tracks_by_album
        USING org.apache.spark.sql.cassandra
        OPTIONS (
          keyspace "music",
          table "tracks_by_album",
          pushdown "true"
        )""")

org.apache.spark.sql.DataFrame = []

### Create a dataframe based on an SQL statement

In [9]:
val track_count_by_year = sqlContext.sql("select 'dummy' as dummy, album_year as year, count(*) as track_count from tmp_tracks_by_album group by album_year")

In [14]:
track_count_by_year show 5

+-----+----+-----------+
|dummy|year|track_count|
+-----+----+-----------+
|dummy|1952|         10|
|dummy|1956|         53|
|dummy|1957|         21|
|dummy|1958|         31|
|dummy|1959|         28|
+-----+----+-----------+



### Register that result as a tmp table

In [17]:
track_count_by_year registerTempTable "tmp_track_count_by_year"

In [19]:
sqlContext.sql("insert into table music.tracks_by_year select dummy, track_count, year from tmp_track_count_by_year")

In [22]:
%%cql select * from music.tracks_by_year limit 10

dummy,track_count,year


### Or ... the one-liner technique. Lets save the data with a simple insert - select statement

In [21]:
%%cql truncate music.tracks_by_year

In [24]:
sqlContext.sql("insert into table music.tracks_by_year select 'dummy' as dummy, count(*) as track_count, album_year from tmp_tracks_by_album group by album_year")

org.apache.spark.sql.DataFrame = [dummy: string, track_count: int, album_year: int]

In [25]:
%%cql select * from music.tracks_by_year limit 10

dummy,track_count,year
dummy,2200,1998
dummy,2170,2001
dummy,2039,1999
dummy,1985,2002
dummy,1980,2000
dummy,1708,1995
dummy,1607,1996
dummy,1604,1997
dummy,1544,1994
dummy,1355,1993
